In [17]:
# Import packages
from fuzzywuzzy import fuzz
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import pymongo
import string
import nltk
import re
import os
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
import warnings
warnings.filterwarnings('ignore')



src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
sys.path.append(src_dir)

# helper functions
from d02_processing.cleaning_signatures import sorted_signatures
from d02_processing.cleaning_signatures import cleaned_signatures
from d01_utils.mongo_cursor_creator import mongo_cursor

# Load the "autoreload" extension
%load_ext autoreload

# reload modules so that as you change code in src, it gets loaded
%autoreload

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
df = pd.read_csv('../../data/02_intermediate//cleaned_and_sorted.csv')
df.drop(columns=['Unnamed: 0'],axis=1, inplace=True)
df = df[pd.notnull(df['products'])]
df.head()

,curl_pattern,density,porosity,texture,products
0,NaN,thin,NaN,NaN,molly didn t want her hair to look pretty that...
1,3a,thin,NaN,NaN,mix of and wash recently changed to using low ...
2,3b,thin,NaN,NaN,mix of and wash recently changed to using low ...
3,3b,thin,normal,NaN,shampoo devacurl nopoo condition detangle gio...
4,3c,thin,normal,NaN,shampoo devacurl nopoo condition detangle gio...


In [15]:
df.products[3]

' shampoo devacurl nopoo condition detangle giovanni smooth as silk leave in the giovanni or mixed chicks only when needed jojoba oil acv wash if you want your life to change you have to change some gs in your life tongue3 '

In [40]:
s = df.products[0]
s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
s = s.lower()
s = re.sub(r"molly" or r"hair", " ", s)
s = re.sub(r"med\s", "medium ", s)
s = re.sub(r"hi\s", "high ", s)
s = re.sub(r"po\s", "porosity ", s)
s = re.sub(r"den\s", "density ", s)

In [41]:
s

'  didn t want her hair to look pretty that was out of the question she only wanted everyone to k she was a clever little girl and not to find fault with her george eliot'

In [20]:
fuzz.ratio( df.products[1], df.products[2])

100

In [50]:
product_list = list(df.products)
len(product_list)

7954

In [48]:
 df.products[i+1]

' slightly growing out my hair currently all one length reaches just above my shoulders former flat iron junkie modified since 9 3 08 new regime california baby tea tree lavender shampoo sulphate free shamp when needed suave tropical coconut condish honey rinse biosilk rock hard gelee plop and add more gel diffuse for more curly look and fuller results i have to work to make my hair curly my hair loves honey plopping and pineapples rule '

In [42]:
ratios = []
for i in range(0,len(product_list)-1):
    ratios.append(fuzz.ratio(str(df.products[i]), str(df.products[i+1])))
    #ratios.append(type(df.products[i]))
ratios    
                 

[3,
 100,
 3,
 100,
 12,
 100,
 3,
 9,
 12,
 100,
 28,
 100,
 24,
 100,
 23,
 100,
 24,
 100,
 40,
 100,
 20,
 1,
 6,
 12,
 10,
 1,
 22,
 100,
 22,
 1,
 100,
 18,
 3,
 100,
 4,
 100,
 3,
 3,
 100,
 19,
 100,
 1,
 24,
 19,
 13,
 4,
 9,
 17,
 100,
 20,
 4,
 9,
 21,
 100,
 2,
 3,
 100,
 23,
 100,
 19,
 100,
 2,
 100,
 18,
 100,
 17,
 100,
 30,
 100,
 25,
 100,
 36,
 100,
 4,
 3,
 4,
 2,
 4,
 100,
 25,
 13,
 22,
 2,
 100,
 1,
 2,
 3,
 3,
 100,
 6,
 13,
 4,
 18,
 20,
 2,
 100,
 19,
 100,
 7,
 2,
 100,
 17,
 100,
 1,
 20,
 23,
 4,
 1,
 100,
 19,
 100,
 18,
 20,
 26,
 20,
 22,
 100,
 3,
 100,
 5,
 100,
 11,
 1,
 1,
 100,
 15,
 5,
 3,
 100,
 8,
 100,
 6,
 100,
 1,
 100,
 29,
 100,
 2,
 100,
 2,
 100,
 14,
 4,
 12,
 22,
 3,
 20,
 100,
 3,
 1,
 100,
 20,
 23,
 1,
 100,
 5,
 9,
 3,
 100,
 20,
 100,
 4,
 100,
 13,
 100,
 1,
 100,
 18,
 100,
 6,
 22,
 2,
 100,
 2,
 7,
 100,
 17,
 23,
 30,
 25,
 11,
 100,
 1,
 19,
 100,
 14,
 100,
 36,
 8,
 5,
 1,
 15,
 100,
 2,
 100,
 4,
 1,
 100,
 10,
 100,
 0,
 1

Top terms per cluster:
Cluster 0:
 hair
 conditioner
 gel
 curl
 curls
 type
 curly
 oil
 length
 leave
Cluster 1:
 attitude
 switch
 lot
 volume
 shine
 perfection
 current
 hydra
 cream
 ogx


Prediction
[0]
[0]
